In [1]:
% matplotlib inline
from __future__ import (division, 
                        print_function)

import os
import sys
import copy
import fnmatch
import warnings
import collections

import numpy as np
import scipy
try:
    from scipy.stats import scoreatpercentile
except:
    scoreatpercentile = False
from scipy.interpolate import interp1d
import cPickle as pickle

# Astropy
from astropy.io import fits
from astropy    import units as u
from astropy.stats import sigma_clip
from astropy.table import Table, Column
from astropy.utils.console import ProgressBar

# Read in the FITS tables for the median profiles

In [12]:
dirProf = 'profiles'

rMbin0 = Table.read(os.path.join(dirProf, 'dr1_redbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin1.fits'))
rMbin1 = Table.read(os.path.join(dirProf, 'dr1_redbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin2.fits'))
rMbin2 = Table.read(os.path.join(dirProf, 'dr1_redbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin3.fits'))

rM100C = Table.read(os.path.join(dirProf, 'dr1_redbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mtotC.fits'))
rM10C = Table.read(os.path.join(dirProf, 'dr1_redbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_minnC.fits'))

nMbin0 = Table.read(os.path.join(dirProf, 'dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin0.fits'))
nMbin1 = Table.read(os.path.join(dirProf, 'dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin1.fits'))
nMbin2 = Table.read(os.path.join(dirProf, 'dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mbin2.fits'))

nM100C = Table.read(os.path.join(dirProf, 'dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_mtotC.fits'))
nM10C = Table.read(os.path.join(dirProf, 'dr1_nonbcg_isedfit_mass_fsps1_sbpsum_imgsub_use_minnC.fits'))

In [13]:
rCol = rMbin0['rkpc'] ** 4.0

In [14]:
rMbin0.colnames

['rkpc',
 'medProfLow',
 'medProfUpp',
 'medProf',
 'avgProfLow',
 'avgProfUpp',
 'avgProf',
 'stdProf']

# Formatting the output string

In [52]:
format1 = "%4.1f & $%5.2f\substack{+%4.2f \\\\ -%4.2f}$ &"
format2 = "$%5.2f\substack{+%4.2f \\\\ -%4.2f}$ &"
format3 = "$%5.2f\substack{+%4.2f \\\\ -%4.2f}$ \\\\"

In [72]:
lastM = 0.0
lastR = 0.0

for ii, rr in enumerate(rCol):
    # redbcg Mbin0
    col2 = rMbin0[ii]['avgProf']
    col2p = (rMbin0[ii]['avgProfUpp'] - rMbin0[ii]['avgProf'])
    col2m = (rMbin0[ii]['avgProf'] - rMbin0[ii]['avgProfLow'])
    # redbcg Mbin1
    col3 = rMbin1[ii]['avgProf']
    col3p = (rMbin1[ii]['avgProfUpp'] - rMbin1[ii]['avgProf'])
    col3m = (rMbin1[ii]['avgProf'] - rMbin1[ii]['avgProfLow'])
    # redbcg Mbin2
    col4 = rMbin2[ii]['avgProf']
    col4p = (rMbin2[ii]['avgProfUpp'] - rMbin2[ii]['avgProf'])
    col4m = (rMbin2[ii]['avgProf'] - rMbin2[ii]['avgProfLow'])
    
    # nonbcg Mbin0
    col5 = nMbin0[ii]['avgProf']
    col5p = (nMbin0[ii]['avgProfUpp'] - nMbin0[ii]['avgProf'])
    col5m = (nMbin0[ii]['avgProf'] - nMbin0[ii]['avgProfLow'])
    # nonbcg Mbin1
    col6 = nMbin1[ii]['avgProf']
    col6p = (nMbin1[ii]['avgProfUpp'] - nMbin1[ii]['avgProf'])
    col6m = (nMbin1[ii]['avgProf'] - nMbin1[ii]['avgProfLow'])
    # nonbcg Mbin2
    col7 = nMbin2[ii]['avgProf']
    col7p = (nMbin2[ii]['avgProfUpp'] - nMbin2[ii]['avgProf'])
    col7m = (nMbin2[ii]['avgProf'] - nMbin2[ii]['avgProfLow'])
    
    #print(rr, col5, last, (col5 - lastM))
    
    if (ii == 0) or ((rr <= 5.0) and 
                     ((rr - lastR) >= 0.3) and 
                     (np.abs(col5 - lastM) > 0.03)) or ((rr <= 20.0) and 
                     ((rr - lastR) >= 1.3) and 
                     (np.abs(col5 - lastM) > 0.05)) or ((rr < 150.0) and 
                     ((rr - lastR) >= 3.5) and 
                     (np.abs(col5 - lastM) > 0.06)):
        string1 = format1 % (rr, col2, col2p, col2m)
        string2 = format2 %(col3, col3p, col3m)
        string3 = format2 %(col4, col4p, col4m)
        string4 = format2 %(col5, col5p, col5m)
        string5 = format2 %(col6, col6p, col6m)
        string6 = format3 %(col7, col7p, col7m)
        print(string1 + string2 + string3 + string4 + string5 + string6)
                     
        lastM = col5
        lastR = rr


 0.0 & $ 9.30\substack{+0.02 \\ -0.02}$ &$ 9.33\substack{+0.02 \\ -0.02}$ &$ 9.34\substack{+0.02 \\ -0.03}$ &$ 9.36\substack{+0.00 \\ -0.00}$ &$ 9.27\substack{+0.00 \\ -0.00}$ &$ 9.32\substack{+0.01 \\ -0.01}$ \\
 0.5 & $ 9.28\substack{+0.02 \\ -0.02}$ &$ 9.31\substack{+0.02 \\ -0.02}$ &$ 9.32\substack{+0.03 \\ -0.02}$ &$ 9.33\substack{+0.00 \\ -0.00}$ &$ 9.25\substack{+0.00 \\ -0.00}$ &$ 9.30\substack{+0.01 \\ -0.01}$ \\
 0.8 & $ 9.25\substack{+0.02 \\ -0.02}$ &$ 9.28\substack{+0.02 \\ -0.02}$ &$ 9.30\substack{+0.02 \\ -0.02}$ &$ 9.29\substack{+0.00 \\ -0.00}$ &$ 9.22\substack{+0.00 \\ -0.00}$ &$ 9.27\substack{+0.01 \\ -0.01}$ \\
 1.2 & $ 9.22\substack{+0.02 \\ -0.02}$ &$ 9.25\substack{+0.02 \\ -0.02}$ &$ 9.28\substack{+0.02 \\ -0.02}$ &$ 9.24\substack{+0.00 \\ -0.00}$ &$ 9.19\substack{+0.00 \\ -0.00}$ &$ 9.24\substack{+0.01 \\ -0.01}$ \\
 1.5 & $ 9.16\substack{+0.02 \\ -0.01}$ &$ 9.20\substack{+0.02 \\ -0.02}$ &$ 9.24\substack{+0.02 \\ -0.02}$ &$ 9.17\substack{+0.00 \\ -0.00}$ &$ 9.1